<a href="https://colab.research.google.com/github/btcnhung1299/tf-practice/blob/master/IMG_HorseVsPeople_TFHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data gathering

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
TRAIN_DIR = "./train_data"
VAL_DIR = "./val_data"
!wget -q https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
!wget -q https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
!unzip -q horse-or-human.zip -d $TRAIN_DIR
!unzip -q validation-horse-or-human -d $VAL_DIR

In [3]:
BATCH_SIZE = 64
raw_shape = (300, 300, 3)

ds_train = ImageDataGenerator(rescale=1./255).flow_from_directory(
    directory=TRAIN_DIR,
    class_mode="binary",
    batch_size=BATCH_SIZE
)
ds_val = ImageDataGenerator(rescale=1./255).flow_from_directory(
    directory=VAL_DIR,
    class_mode="binary",
    batch_size=BATCH_SIZE
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Transfer learning

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers.experimental.preprocessing import Resizing

### ResNet-50

The default size of input image is 224 x 224, but not fixed.

In [5]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4" 

resnet = Sequential()
resnet.add(Resizing(150, 150, input_shape=raw_shape))
resnet.add(hub.KerasLayer(resnet_url, trainable=False))
resnet.add(Dense(1, activation="sigmoid"))
resnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 150, 150, 3)       0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 2048)              23564800  
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [6]:
resnet.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

In [7]:
resnet_history = resnet.fit(ds_train, epochs=10, validation_data=ds_val)

Epoch 1/10
17/17 [==============================] - 9s 537ms/step - loss: 0.3171 - acc: 0.8549 - val_loss: 0.0547 - val_acc: 1.0000
Epoch 2/10
17/17 [==============================] - 8s 482ms/step - loss: 0.0193 - acc: 0.9981 - val_loss: 0.0198 - val_acc: 1.0000
Epoch 3/10
17/17 [==============================] - 8s 480ms/step - loss: 0.0091 - acc: 0.9990 - val_loss: 0.0120 - val_acc: 1.0000
Epoch 4/10
17/17 [==============================] - 8s 478ms/step - loss: 0.0073 - acc: 0.9981 - val_loss: 0.0098 - val_acc: 1.0000
Epoch 5/10
17/17 [==============================] - 8s 480ms/step - loss: 0.0057 - acc: 0.9990 - val_loss: 0.0105 - val_acc: 1.0000
Epoch 6/10
17/17 [==============================] - 8s 478ms/step - loss: 0.0049 - acc: 0.9990 - val_loss: 0.0084 - val_acc: 1.0000
Epoch 7/10
17/17 [==============================] - 8s 483ms/step - loss: 0.0041 - acc: 1.0000 - val_loss: 0.0077 - val_acc: 1.0000
Epoch 8/10
17/17 [==============================] - 8s 484ms/step - loss: 0.

### MobileNetV2

Size of input image is 224 x 224, fixed.

In [8]:
mobilenet_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4" 

mobilenet = Sequential()
mobilenet.add(Resizing(224, 224, input_shape=raw_shape))
mobilenet.add(hub.KerasLayer(mobilenet_url, trainable=False))
mobilenet.add(Dense(1, activation="sigmoid"))
mobilenet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing_1 (Resizing)        (None, 224, 224, 3)       0         
_________________________________________________________________
keras_layer_1 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [9]:
mobilenet.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

In [10]:
mobilenet_history = mobilenet.fit(ds_train, epochs=10, validation_data=ds_val)

Epoch 1/10
17/17 [==============================] - 8s 493ms/step - loss: 0.3416 - acc: 0.8851 - val_loss: 0.1609 - val_acc: 0.9883
Epoch 2/10
17/17 [==============================] - 8s 466ms/step - loss: 0.0693 - acc: 0.9971 - val_loss: 0.0707 - val_acc: 0.9961
Epoch 3/10
17/17 [==============================] - 8s 463ms/step - loss: 0.0358 - acc: 0.9971 - val_loss: 0.0496 - val_acc: 0.9961
Epoch 4/10
17/17 [==============================] - 8s 462ms/step - loss: 0.0258 - acc: 0.9971 - val_loss: 0.0436 - val_acc: 0.9961
Epoch 5/10
17/17 [==============================] - 8s 465ms/step - loss: 0.0209 - acc: 0.9971 - val_loss: 0.0318 - val_acc: 0.9961
Epoch 6/10
17/17 [==============================] - 8s 462ms/step - loss: 0.0170 - acc: 0.9971 - val_loss: 0.0301 - val_acc: 0.9961
Epoch 7/10
17/17 [==============================] - 8s 458ms/step - loss: 0.0145 - acc: 0.9971 - val_loss: 0.0277 - val_acc: 0.9961
Epoch 8/10
17/17 [==============================] - 8s 456ms/step - loss: 0.